

---



---


#**Solución**


---

#**Importe de Librerias**

In [ ]:
# @title
import warnings

import numpy as np
import astropy.constants as c
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp, quad



---

#**Definición de Funciones**

***Nota**: Tomado del Notebook de Jorge Zuluaga

En esta celda se definen las siguientes funciones:
- `lane_emden`: Función de Ecuación Diferencial de **Lane - Emden**
- `surface`: Función de evaluación de Singularidades
- `politropa`: Función integrada con `lane_emden` y `surface` para realizar la solución para cualquier valor de `gamma
- `luminosity_model`: Función integrada con `politropa` para establecer el valor de Luminosidad

In [ ]:
# @title
def lane_emden(x, y, gamma):

  theta, theta_p = y

  dtheta = theta_p
  dtheta_p = -2*theta_p/x - abs(theta)**(1/(gamma-1))

  return np.array([dtheta, dtheta_p])


############################################################
def surface(t, y, gamma):

  theta, theta_p = y

  return theta


###########################################################
def politropa(gamma, x_max = 10):

  global surface

  xs = np.linspace(1e-4, x_max, 1000)
  y0 = [1,0]

  #Resolvemos
  sol = solve_ivp(lane_emden, [xs[0], xs[-1]], t_eval = xs, y0 = y0,
                  args = (gamma,), events = surface)

  thetas = sol.y[0]
  theta_ps = sol.y[1]

  #Buscamos superficie
  if len(sol.t_events[0]) > 0:

    hasurf = True
    x_surf = sol.t_events[0][0]
    theta_surf = sol.y_events[0][0,0]
    theta_p_surf = sol.y_events[0][0,1]

  else:

    hasurf = False
    x_surf = xs[-1]
    theta_surf = thetas[-1]
    theta_p_surf = theta_ps[-1]

  #Interpolamos
  xns = np.linspace(xs[0], x_surf, 100)
  theta_ns = np.interp(xns, xs, thetas)
  theta_pns = np.interp(xns, xs, theta_ps)

  return hasurf, xns, theta_ns, theta_pns


#####################################################
def luminosity_model(gamma, args):

  #Argumentos Globales de la Estrella
  Mmol, Ms, Rs, e0, To, n = args

  #R = c.R.value
  #G = c.G.value
  R = 8.31446261815324 #J/kg/mol
  G = 6.67e-11 # m^3/(kg s^2)
  cp = gamma/(gamma-1)*R/Mmol

  # Propiedades de la Estrella
  g0 = G*Ms/Rs**2
  T0 = g0*Rs/cp

  # Solución a la ecuación de Lane-Emden
  hasurf, xns, tetans, tetapns = politropa(gamma)

  x0 = xns[-1]
  tetap0 = tetapns[-1]
  lamb = Rs/x0

  # Calculo de temperatura y densidad central
  Tc = -T0/(x0*tetap0)
  rhoc = cp*Tc/(4*np.pi*G*lamb**2)

  # Perfil de temperatura y densidad
  rs = xns*lamb
  Ts = tetans*Tc
  rhos = (Tc*rhoc**(1-gamma)/Ts)**(1/(1-gamma))

  #Tasa de producción de energia
  es = e0*((Ts/To)**n)

  #Eliminamos Precaución
  warnings.filterwarnings('ignore')

  # Calculo de la Luminosidad total
  rho_fun = interp1d(rs,rhos)
  e_fun = interp1d(rs, es)
  integrando = lambda r: 4*np.pi*r**2*rho_fun(r)*e_fun(r)
  L, dL = quad(integrando,xns[0]*Rs,Rs)
  return L, Tc, rhoc



---

#**Ejercicio 1**

In [ ]:
# @title
#Parametros del Sol
X = 0.73
Z = 0.02
Mmol = 4/(3+5*X-Z)*1e-3

#Ms = c.M_sun.value
#Rs = c.R_sun.value
Ms = 2e30 # kg
Rs = 7.96e8 #m

#Tasa de Producción de Energía
e0 = 0.0125
To = 15e6
n = 4

args_sun = [Mmol, Ms, Rs, e0, To, n]

In [ ]:
# @title
L5_3 = luminosity_model(5/3, args_sun)[0]
L4_3 = luminosity_model(4/3, args_sun)[0]

print(f"Gamma: 5/3\nLuminosidad Total: {L5_3/c.L_sun.value} L_sun")
print(f"Gamma: 4/3\nLuminosidad Total: {L4_3/c.L_sun.value} L_sun")

Gamma: 5/3
Luminosidad Total: 0.43107608098325556 L_sun
Gamma: 4/3
Luminosidad Total: 3.301319854533529 L_sun




---


#**Ejercicio 2**

El método para obtener el valor de `gamma` tal que el valor de Luminosidad sea el más cercano al Sol se fundamenta en minimizar la función *Diferencia absoluta de Luminosidades* tal que al ejecutar `scipy.optimize.minimize` se logre obtener el valor más cercano de `gamma` con el fin de que la función *Diferencia* sea cero.

In [ ]:
# @title
#Función de Minimizacion
L_min = lambda gamma, args: abs(c.L_sun.value - luminosity_model(gamma, args)[0])

#Minimizamos
gamma_min = minimize_scalar(L_min, [5/3,4/3], args = (args_sun,)).x

print(f'El valor de Gamma que minimiza la Luminosidad es:\nGamma = {gamma_min}')

El valor de Gamma que minimiza la Luminosidad es:
Gamma = 1.443419855136474




---

#**Ejercicio 3**

Aplicamos misma metodología al Literal anterior

Se toma valor de Z = 0.03 [1] y se asume fracción de masa de Hidrogeno como el Sol

1. Martin Beech, Corey McCowan, Lowell Peltier. The Composition and Evolutionary Status of Proxima Centauri. American Journal of
Astronomy and Astrophysics. Vol. 5, No. 1, 2017, pp. 1-5. doi: 10.11648/j.ajaa.20170501.11

In [ ]:
# @title
#Parametros para Proxima Centauri
X = 0.73
Z = 0.03
Mmol = 4/(3+5*X-Z)*1e-3

Ms = 0.123*c.M_sun.value
Rs = 0.145*c.R_sun.value

#Tasa de Producción de Energía
e0 = 0.00003875
To = 8e6
n = 5

args_prox = [Mmol, Ms, Rs, e0, To, n]

In [ ]:
# @title

L_min = lambda gamma, args: abs(0.0017*c.L_sun.value - luminosity_model(gamma, args)[0])

#Minimizamos
gamma_min = minimize_scalar(L_min, [1.25, 2], args = (args_prox,)).x

print(f'El valor de Gamma que minimiza la Luminosidad de Proxima Centauri es:\nGamma = {gamma_min}')

El valor de Gamma que minimiza la Luminosidad de Proxima Centauri es:
Gamma = 1.5376373254787707


In [ ]:
# @title
T_prox, rho_prox = luminosity_model(gamma_min, args_prox)[1:]

print(f'Los valores de Temperatura y Densidad Central que predice el modelo son:\nT = {T_prox} K\nrho = {rho_prox} kg/m³')

Los valores de Temperatura y Densidad Central que predice el modelo son:
T = 6849664.973401623 K
rho = 540949.7078978337 kg/m³
